# Extended Portfolio Optimization Problem

## Import Libraries

In [1]:
import sys, os

if 'google.colab' in sys.modules:
    %pip install idaes-pse --pre >/dev/null 2>/dev/null
    !idaes get-extensions --to ./bin 
    os.environ['PATH'] += ':bin'
    solver = "ipopt"
else:
    solver = "mosek_direct"

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Preprocessing

In [2]:
# Set the seed for reproducibility
np.random.seed(5)

n = 10

# Generate mean of the portfolio pbar
pbar = np.ones((n, 1)) * 0.03 + np.vstack((np.random.rand(n-1, 1), np.array([[0]]))) * 0.12

# Generate covariance matrix of the portfolio Sigma
Sigma = np.random.randn(n, n)
Sigma = np.dot(Sigma.T, Sigma)
Sigma = Sigma / np.max(np.abs(np.diag(Sigma))) * 0.2
Sigma[:, n-1] = np.zeros(n)
Sigma[n-1, :] = np.zeros((1, n))

# If you want to verify the positive semidefiniteness of the matrix Sigma
# Calculate the eigenvalues of S
eigenvalues = np.linalg.eigvals(Sigma)

# Check if all eigenvalues are non-negative
is_positive_semidefinite = np.all(eigenvalues >= 0)
print("Is Sigma positive semidefinite?", is_positive_semidefinite)



Is Sigma positive semidefinite? True


## Construct the Model

In [3]:
def portfolio(n, pbar, Sigma):
    # Create a model
    m = pyo.ConcreteModel("Extended Portfolio Optimization Problem")

    # Sets of the index
    m.i = pyo.RangeSet(0, n-1)

    # Decision variables
    m.x_long = pyo.Var(range(n), domain=pyo.NonNegativeReals)
    m.x_short = pyo.Var(range(n), domain=pyo.NonNegativeReals)

    # Objective function
    @m.Objective(sense=pyo.minimize)
    def objective(m):
        return sum(Sigma[i, j] * (m.x_long[i] - m.x_short[i]) * (m.x_long[j] - m.x_short[j]) for i in m.i for j in m.i)

    # Constraints
    @m.Constraint()
    def min_return_rate(m):
        return sum(pbar[i] * (m.x_long[i] - m.x_short[i]) for i in range(n)) >= 0.1

    @m.Constraint()
    def total_sum(m):
        return sum((m.x_long[i] - m.x_short[i]) for i in m.i) == 1

    @m.Constraint()
    def total_short_position(m):
        return sum(m.x_short[i] for i in m.i) <= 0.5

    # return the finished model
    return m

## Solve the Model

In [4]:
def solve_model(model):
    SOLVER.solve(model)
    model.display()

In [5]:
model = portfolio(n, pbar, Sigma)
solve_model(model)

Model Extended Portfolio Optimization Problem

  Variables:
    x_long : Size=10, Index=x_long_index
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
          0 :     0 : 0.002322619971796285 :  None : False : False : NonNegativeReals
          1 :     0 :   0.2688646872409641 :  None : False : False : NonNegativeReals
          2 :     0 : 0.005174864425589278 :  None : False : False : NonNegativeReals
          3 :     0 :  0.22439480832765085 :  None : False : False : NonNegativeReals
          4 :     0 :  0.14891903312994378 :  None : False : False : NonNegativeReals
          5 :     0 :    0.189021210001199 :  None : False : False : NonNegativeReals
          6 :     0 :   0.2050592847780979 :  None : False : False : NonNegativeReals
          7 :     0 : 0.005316021350962623 :  None : False : False : NonNegativeReals
          8 :     0 : 0.005440367250741802 :  None : False : False : NonNegativeReals
          9 :     0 :    0.437829806052645 :  Non

## Result Visualization

In [6]:
def visualization(model):
    x_long_values = [pyo.value(model.x_long[i]) for i in model.i]
    x_short_values = [pyo.value(model.x_short[i]) for i in model.i]
    optimal_value = pyo.value(model.objective)

    assets = [f'Asset {i}' for i in range(1, n+1)]
    df = pd.DataFrame({
        'x_long': x_long_values,
        'x_short': x_short_values
    }, index=assets)
    return df, optimal_value

result = visualization(model)
df = result[0]
df

,x_long,x_short
Asset 1,0.002323,0.039715
Asset 2,0.268865,0.005403
Asset 3,0.005175,0.049096
Asset 4,0.224395,0.005507
Asset 5,0.148919,0.005005
Asset 6,0.189021,0.005062
Asset 7,0.205059,0.005327
Asset 8,0.005316,0.268624
Asset 9,0.005440,0.103361
Asset 10,0.437830,0.005244


In [7]:
print(f"The optimal value is {result[1]}. ")

The optimal value is 0.0022284409481733466. 
